In [16]:
import pandas as pd
#import numpy as np



#* PassengerId => 乘客ID 
#* Pclass => 乘客等级(1/2/3等舱位) 
#* Name => 乘客姓名 
#* Sex => 性别 
#* Age => 年龄 
#* SibSp => 堂兄弟/妹个数 
#* Parch => 父母与小孩个数 
#* Ticket => 船票信息 
#* Fare => 票价 
#* Cabin => 客舱 
#* Embarked => 登船港口

# 分别读取训练数据和测试数据,文件路径最后那里是两个斜杠，因为后面是t字母 会被转义
train = pd.read_csv('D:\soft\python_tools\kaggle\Titanic\\train.csv')
test = pd.read_csv('D:\soft\python_tools\kaggle\Titanic\\test.csv')

#先分别输出训练、测试数据的基本信息
print train.info()
print test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null 

In [17]:
#选取对预测有效的特征
selected_features = ['Pclass','Sex','Age','Embarked','SibSp','Parch','Fare']

X_train = train[selected_features]
X_test = test[selected_features]

Y_train = train['Survived']

#通过观察得知，Embarked字段存在缺失值，需要补充
print X_train['Embarked'].value_counts()
print X_test['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64
S    270
C    102
Q     46
Name: Embarked, dtype: int64


In [18]:
#对于Embarked这种类型的特征，我们使用出现频率最高的特征值来填充，是一种相对减少引入误差的方法
X_train['Embarked'].fillna('S',inplace=True)
X_test['Embarked'].fillna('S',inplace=True)

#对于age这种类型的特征，一般是采用平均值或者是中位数来填充缺失值，也是一种相对减少引入误差的方法
X_train['Age'].fillna(X_train['Age'].mean(),inplace=True)
X_test['Age'].fillna(X_test['Age'].mean(),inplace=True)
X_test['Fare'].fillna(X_test['Fare'].mean(),inplace=True)

X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
Embarked    891 non-null object
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 48.8+ KB


In [28]:
#采用DictVectorizer对特征进行向量化
from sklearn.feature_extraction import DictVectorizer
dict_vec = DictVectorizer(sparse=False)
X_train = dict_vec.fit_transform(X_train.to_dict(orient='record'))
X_test = dict_vec.transform(X_test.to_dict(orient='record'))
dict_vec.feature_names_

AttributeError: 'numpy.ndarray' object has no attribute 'to_dict'

In [27]:

from sklearn.ensemble import RandomForestClassifier

#使用默认配置初始化RandomForestClassifier
rfc = RandomForestClassifier()

#从流行工具包xgboost导入XGBClassifier用于处理分类预测问题，目前xgboost模块尚未导入
#from xgboost import XGBClassifier
#也是用默认配置初始化XGBClassifier
#xgbc = XGBClassifier()

from sklearn.cross_validation import cross_val_score

#使用5折交叉验证的方法在训练集上分别对默认配置的 RandomForestClassifier 以及 XGBClassifier进行性能评估，并获得平均分类的准确性得分
cross_val_score(rfc, X_train, Y_train, cv=5).mean()

0.80254636254440503

In [31]:
#使用默认的RandomForestClassifier进行预测操作
rfc.fit(X_train,Y_train)
rfc_y_predict = rfc.predict(X_test)
rfc_submission = pd.DataFrame({'PassengerId': test['PassengerId'],'Survived' : rfc_y_predict})

#将预测结果存储在rfc_submission_csv中
rfc_submission.to_csv('D:\soft\python_tools\kaggle\Titanic\\rfc_submission.csv', index=False)
